In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Задания к уроку 5

### 1. Задание:

###### Вектор – это частный случай матрицы 1хN и Nх1. Повторите материал для векторов, уделяя особое внимание умножению A∙B.

In [17]:
a = np.array([[1, 2, 3, 4, 5, 6]], float)
print(a)
print(a[0, 2])
print(a.shape)

[[1. 2. 3. 4. 5. 6.]]
3.0
(1, 6)


In [18]:
b = a * 2
print(b)

c = a + 1
print(c)

d = c + b
print(d)

d = b - c
print(d)


[[ 2.  4.  6.  8. 10. 12.]]
[[2. 3. 4. 5. 6. 7.]]
[[ 4.  7. 10. 13. 16. 19.]]
[[0. 1. 2. 3. 4. 5.]]


In [19]:
f = d.T
print(d)

print("")
print(f)

[[0. 1. 2. 3. 4. 5.]]

[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]]


In [24]:
i, j = 0, 0
print(d[i, 0] * f[0, j] + d[i, 1] * f[1, j] + d[i, 2] * f[2, j] + d[i, 3] * f[3, j] + d[i, 4] * f[4, j] + d[i, 5] * f[5, j])

g = np.dot(d, f)
print(g)
d.shape, f.shape

55.0
[[55.]]


((1, 6), (6, 1))

In [28]:
i, j = 5, 5
print(f[i, 0] * d[0, j])

g = np.dot(f, d)
print(g)
f.shape, d.shape

25.0
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  1.  2.  3.  4.  5.]
 [ 0.  2.  4.  6.  8. 10.]
 [ 0.  3.  6.  9. 12. 15.]
 [ 0.  4.  8. 12. 16. 20.]
 [ 0.  5. 10. 15. 20. 25.]]


((6, 1), (1, 6))

###### Вычислите, по возможности не используя программирование: (5Е)–1, где Е – единичная матрица размера 5х5.

Пусть Е – единичная матрица размера 5х5:
### $E =
\begin{pmatrix}
1 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 1
\end{pmatrix}$
Тогда P – матрица размера 5х5 равная 5E:
### $P =
\begin{pmatrix}
5 & 0 & 0 & 0 & 0\\
0 & 5 & 0 & 0 & 0\\
0 & 0 & 5 & 0 & 0\\
0 & 0 & 0 & 5 & 0\\
0 & 0 & 0 & 0 & 5
\end{pmatrix}$
### $P \cdot E = E \cdot P = P$
### $P \cdot P^{-1} = E$
### $P^{-1} = \frac{E}{P} = \frac{E}{5E} = \frac{1}{5} E$
– деление, как и умножение, единичных матриц дают единичную матрицу, тогда обратная матрица для P равна:
### $P^{-1} =
\begin{pmatrix}
1/5 & 0 & 0 & 0 & 0\\
0 & 1/5 & 0 & 0 & 0\\
0 & 0 & 1/5 & 0 & 0\\
0 & 0 & 0 & 1/5 & 0\\
0 & 0 & 0 & 0 & 1/5
\end{pmatrix}$
Проверим вычислениями на Питоне:

In [32]:
e = np.identity(5)
print(e)
p = 5*e
print(p)
print(np.linalg.inv(p))

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[[5. 0. 0. 0. 0.]
 [0. 5. 0. 0. 0.]
 [0. 0. 5. 0. 0.]
 [0. 0. 0. 5. 0.]
 [0. 0. 0. 0. 5.]]
[[ 0.2  0.   0.   0.   0. ]
 [ 0.   0.2  0.   0.   0. ]
 [ 0.   0.   0.2  0.   0. ]
 [-0.  -0.  -0.   0.2 -0. ]
 [ 0.   0.   0.   0.   0.2]]


### 2. Задание:

###### Вычислите определитель:
# $
\left|\begin{matrix}
1 & 2 & 3\\
4 & 0 & 6\\
7 & 8 & 9
\end{matrix}
\right|
$

## $\det A = \left|
\begin{array}{cccc}
1 & 2 & 3\\
4 & 0 & 6\\
7 & 8 & 9
\end{array}
\right|=
-4\left| 
\begin{array}{cccc}
2 & 3\\ 
8 & 9
\end{array} 
\right|
+0\left| 
\begin{array}{cccc}
1 & 3\\ 
7 & 9
\end{array} 
\right|
-6\left| 
\begin{array}{cccc}
1 & 2\\ 
7 & 8
\end{array} 
\right|=60
$


In [35]:
- 4 * (2 * 9 - 3 * 8) + 0 * (1 * 9 - 3 * 7) - 6 * (1 * 8 - 2 * 7)

60

Проверим вычислениями на Питоне:

In [36]:
a = np.array([[1, 2, 3], [4, 0, 6], [7, 8, 9]])
np.linalg.det(a)

59.999999999999986

### 3. Задание:

###### 1. Вычислите матрицу, обратную данной:
# $
\left|\begin{matrix}
1 & 2 & 3\\
4 & 0 & 6\\
7 & 8 & 9
\end{matrix}
\right|
$

Определитель найден в предыдущем задании:
## $\det A = \left|
\begin{array}{cccc}
1 & 2 & 3\\
4 & 0 & 6\\
7 & 8 & 9
\end{array}
\right|=
-4\left| 
\begin{array}{cccc}
2 & 3\\ 
8 & 9
\end{array} 
\right|
+0\left| 
\begin{array}{cccc}
1 & 3\\ 
7 & 9
\end{array} 
\right|
-6\left| 
\begin{array}{cccc}
1 & 2\\ 
7 & 8
\end{array} 
\right|=60
$

Найдем матрицу миноров M, вычислив их для каждого элемента матрицы A:
### $m_{00} = \left| 
\begin{array}{cccc}
0 & 6\\ 
8 & 9
\end{array} 
\right| = 0\cdot 9 - 6\cdot 8
= -48
, m_{01} = \left| 
\begin{array}{cccc}
4 & 6\\ 
7 & 9
\end{array} 
\right| = 4\cdot 9 - 6\cdot 7
= -6
, m_{02} = \left| 
\begin{array}{cccc}
4 & 0\\ 
7 & 8
\end{array} 
\right| = 4\cdot 8 - 0\cdot 7
= 32
\\m_{10} = \left| 
\begin{array}{cccc}
2 & 3\\ 
8 & 9
\end{array} 
\right| = 2\cdot 9 - 3\cdot 8
= -6
, m_{11} = \left| 
\begin{array}{cccc}
1 & 3\\ 
7 & 9
\end{array} 
\right| = 1\cdot 9 - 3\cdot 7
= -12
, m_{12} = \left| 
\begin{array}{cccc}
1 & 2\\ 
7 & 8
\end{array} 
\right| = 1\cdot 8 - 2\cdot 7
= -6
\\ m_{20} = \left| 
\begin{array}{cccc}
2 & 3\\ 
0 & 6
\end{array} 
\right| = 2\cdot 6 - 3\cdot 0
= 12
, m_{21} = \left| 
\begin{array}{cccc}
1 & 3\\ 
4 & 6
\end{array} 
\right| = 1\cdot 6 - 3\cdot 4
= -6
, m_{22} = \left| 
\begin{array}{cccc}
1 & 2\\ 
4 & 0
\end{array} 
\right| = 1\cdot 0 - 2\cdot 4
= -8
$
## $ M = \left|
\begin{array}{cccc}
-48 & -6 & 32\\
-6 & -12 & -6\\
12 & -6 & -8
\end{array}
\right|
$

Найдем также матрицу алгебраических дополнений и транспонируем ее:
## $ A_* = \left|
\begin{array}{cccc}
-48 & 6 & 32\\
6 & -12 & 6\\
12 & 6 & -8
\end{array}
\right|
$
## $ A_*^T = \left|
\begin{array}{cccc}
-48 & 6 & 12\\
6 & -12 & 6\\
32 & 6 & -8
\end{array}
\right|
$

Наконец вычислим обратную матрицу для матрицы A:
## $ A^{-1} 
= \frac{A_*^T}{\det A}
= \frac{1}{60}\left|
\begin{array}{cccc}
-48 & 6 & 12\\
6 & -12 & 6\\
32 & 6 & -8
\end{array}
\right|
= \left|
\begin{array}{cccc}
-0.8 & 0.1 & 0.2\\
0.1 & -0.2 & 0.1\\
0.53 & 0.1 & -0.13
\end{array}
\right|
$

Также проверим результат через умножение матрицы $A$ на обратную матрицу $A^{-1}$ в результате должна получиться единичная матрица:
## $A \cdot A^{-1}
=
\left|
\begin{array}{cccc}
1 & 2 & 3\\
4 & 0 & 6\\
7 & 8 & 9
\end{array}
\right|
\cdot
\left|
\begin{array}{cccc}
-0.8 & 0.1 & 0.2\\
0.1 & -0.2 & 0.1\\
0.53 & 0.1 & -0.13
\end{array}
\right|
=
$
### $
=
\left|
\begin{array}{cccc}
1*-0.8+2*0.1+3*0.53 & 1*0.1+2*-0.2+3*0.1 & 1*0.2+2*0.1+3*-0.13\\
4*-0.8+0*0.1+6*0.53 & 4*0.1+0*-0.2+6*0.1 & 4*0.2+0*0.1+6*-0.13\\
7*-0.8+8*0.1+9*0.53 & 7*0.1+8*-0.2+9*0.1 & 7*0.2+8*0.1+9*-0.13
\end{array}
\right|
=
$
## $
= \left|
\begin{array}{cccc}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{array}
\right|
= E
$


Проверим вычислениями на Питоне:

In [61]:
b = np.linalg.inv(a)
print(b)
np.dot(a, b)

[[-0.8         0.1         0.2       ]
 [ 0.1        -0.2         0.1       ]
 [ 0.53333333  0.1        -0.13333333]]


array([[ 1.00000000e+00, -2.77555756e-17,  5.55111512e-17],
       [ 0.00000000e+00,  1.00000000e+00,  1.11022302e-16],
       [ 0.00000000e+00,  2.77555756e-17,  1.00000000e+00]])

###### 2. Приведите пример матрицы 4х4, ранг которой равен 1.

Примером матрицы с рангом 1 может быть любая квадратная матрица с одинаковыми ненулевыми элементами (миноры первого ранга будут все ненулевые, так как это элементы матрицы, а все миноры второго и выше ранга будут нулевые, так как определители матриц с одинаковыми элементами всегда нулевые) :
## $
\left|
\begin{array}{cccc}
5 & 5 & 5 & 5\\
5 & 5 & 5 & 5\\
5 & 5 & 5 & 5\\
5 & 5 & 5 & 5
\end{array}
\right|
$
## $
M_1 = 5\\
M_2 = \left| 
\begin{array}{cccc}
5 & 5\\ 
5 & 5
\end{array}
\right|
= 5*5 -5*5 = 0
$

Проверим вычислениями на Питоне:

In [66]:
c = np.array([[5, 5, 5, 5], [5, 5, 5, 5], [5, 5, 5, 5], [5, 5, 5, 5]])
print(c)
np.linalg.matrix_rank(c, 0.0001)

[[5 5 5 5]
 [5 5 5 5]
 [5 5 5 5]
 [5 5 5 5]]


1

### 4. Задание:

###### Вычислите скалярное произведение двух векторов:  (1, 5)  и  (2, 8)


_Вариант 1 (через сумму произведений координат)_
### $
S = 1 \cdot 2 + 5 \cdot 8 = 42
$
_Вариант 2 (через произведение норм и косинуса угла)_
### $(\vec a \cdot \vec b)  = a \cdot b  \cdot \cos \alpha =\\
= \sqrt{1^2 + 5^2} \cdot \sqrt{2^2 + 8^2} \cdot \cos|\alpha_a - \alpha_b| =\\
= 5.10 \cdot 8.25 \cdot \cos|\alpha_a - \alpha_b| =\\
= 42.048 \cdot \cos|\arctan\frac{5}{1} - \arctan\frac{8}{2}| = \\
= 42.048 \cdot 0.999 = 42
$

Проверим вычислениями на Питоне:

In [80]:
# Вариант 1 (через сумму произведений координат)
a = np.array([1, 5], float)
b = np.array([2, 8], float)
s = np.inner(a, b)
print(s)
# Вариант 2 (через произведение норм и косинуса угла)
s = np.linalg.norm(a) * np.linalg.norm(b) * np.cos(abs(np.arctan(5/1) - np.arctan(8/2)))
print(s)

42.0
42.0


### 5. Задание:

###### Вычислите смешанное произведение трех векторов: (1, 5, 0), (2, 8, 7) и (7, 1.5, 3) 


In [74]:
a = np.array([1, 5, 0], float)
b = np.array([2, 8, 7], float)
c = np.array([7, 1.5, 3], float)
v = np.cross(a, b)
print (np.inner(v, c))
w = np.cross(b, c)
print (np.inner(w, a))

228.5
228.5


# Задания к уроку 6

### 1. Задание:

Решите линейную систему:
# $
\left[\begin{matrix}
1 & 2 & 3\\
4 & 0 & 6\\
7 & 8 & 9
\end{matrix}
\right]\cdot X = \left[\begin{matrix}
12\\
2\\
1
\end{matrix}
\right]
$

### 2. Задание:

Найдите псевдорешение:
# $x + 2y - z = 1\\
3x - 4y = 7\\
8x - 5y + 2z = 12\\
2x - 5z = 7\\
11x + 4y - 7z = 15
$

### 3. Задание:

Сколько решений имеет линейная система:
# $
\left[\begin{matrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{matrix}
\right]\cdot X = \left[\begin{matrix}
12\\
2\\
1
\end{matrix}
\right]
$

Если ноль – то измените вектор правой части так, чтобы система стала совместной, и решите ее.

### 4. Задание:

 Вычислите LU-разложение матрицы:
 # $
\left[\begin{matrix}
1 & 2 & 3\\
2 & 16 & 21\\
4 & 28 & 73
\end{matrix}
\right]
$

После этого придумайте вектор правых частей и решите полученную линейную систему трех уравнений с данной матрицей.


### 5. Задание:

Найдите нормальное псевдорешение недоопределенной системы:
# $
x + 2y - z = 1\\
8x - 5y + 2z = 12
$
Для этого определите функцию Q(x,y,z), равную норме решения, и найдите ее минимум. 


### 6. Задание:

Найдите одно из псевдорешений вырожденной системы:
# $
\left[\begin{matrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{matrix}
\right]\cdot X = \left[\begin{matrix}
2\\
5\\
11
\end{matrix}
\right]
$


Попробуйте также отыскать и нормальное псевдорешение.